In [1]:
import gspread
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
service_account = gspread.service_account(filename='./config/expense-tracker-358105-199d116b0a6d.json')
workbook = service_account.open('badminton_tracking')
worksheet = workbook.get_worksheet(0)

In [3]:
df = pd.DataFrame(worksheet.get_all_records()).drop(["Timestamp", "result"], axis=1)

df.columns = ["date", "team_1_player_1", "team_1_player_2", "team_2_player_1", "team_2_player_2", "points_team_1", "points_team_2", "venue"]

df['winner'] = np.where(df.points_team_1 > df.points_team_2, 'team_1', 'team_2')
df['margin'] = abs(df.points_team_1 - df.points_team_2)
df['total_points_per_game'] = df["points_team_1"] + df["points_team_2"]

df = df.applymap(lambda x: f'{x}'.lower().strip() if isinstance(x, str) else x)

In [4]:
df['point_bins'] = pd.cut(
    df['total_points_per_game'],
    [0, 30, 35, 40, 45, float("inf")],
    right=False,
    labels=['< 30', '30 - 35', '35 - 40', '40 - 45', "> 45"]
)

In [5]:
players_list = list(np.unique(df[['team_1_player_1', 'team_1_player_2', 'team_2_player_1', 'team_2_player_2']].values))

In [6]:
def get_player_stats(player, df):
    player_matches = df[
        np.where(
            np.logical_or.reduce([df[i] == player for i in ["team_1_player_1", "team_1_player_2", "team_2_player_1", "team_2_player_2"]]),
            True,
            False
        )
    ].copy()
    
    player_matches["belongs_to"] = np.where(
        np.logical_or(
            *[player_matches[i] == player for i in ["team_1_player_1", "team_1_player_2"]]
        ),
        'team_1',
        'team_2'
    )
    
    player_matches['player_team_points'] = np.where(
        player_matches["belongs_to"] == 'team_1',
        player_matches["points_team_1"],
        player_matches["points_team_2"]
    )
    
    player_matches['result'] = np.where(player_matches.belongs_to == player_matches.winner, "win", "loss")
    player_matches['is_win'] = np.where(player_matches.result == "win", 1, 0)
    
    return player_matches

In [7]:
team_1 = ['raghotham', 'nithin']
team_2 = ['ajay', 'vinay']

In [12]:
player_matches = get_player_stats(team_1[0], df)
player_matches['partner'] = np.where(
    player_matches["belongs_to"] == 'team_1',
    np.where(
        player_matches["team_1_player_1"] == team_1[0],
        player_matches["team_1_player_2"],
        player_matches["team_1_player_1"]
    ),
    np.where(
        player_matches["team_2_player_1"] == team_1[0],
        player_matches["team_2_player_2"],
        player_matches["team_2_player_1"]
    ),
)

In [13]:
player_matches

,date,team_1_player_1,team_1_player_2,team_2_player_1,team_2_player_2,points_team_1,points_team_2,venue,winner,margin,total_points_per_game,point_bins,belongs_to,player_team_points,result,is_win,partner
0,12/27/2022,vinay,raghotham,ajay,aakarsh,15,21,isro match point,team_2,6,36,35 - 40,team_1,15,loss,0,vinay
1,12/28/2022,vinay,raghotham,ajay,sameer,25,27,isro match point,team_2,2,52,> 45,team_1,25,loss,0,vinay
2,12/28/2022,vinay,raghotham,ajay,srinidhi,21,18,isro match point,team_1,3,39,35 - 40,team_1,21,win,1,vinay
3,12/28/2022,sameer,raghotham,vinay,srinidhi,14,21,isro match point,team_2,7,35,35 - 40,team_1,14,loss,0,sameer
4,12/28/2022,sameer,raghotham,ajay,srinidhi,21,18,isro match point,team_1,3,39,35 - 40,team_1,21,win,1,sameer
5,12/28/2022,sameer,raghotham,ajay,vinay,24,26,isro match point,team_2,2,50,> 45,team_1,24,loss,0,sameer
6,12/28/2022,srinidhi,raghotham,ajay,vinay,19,21,isro match point,team_2,2,40,40 - 45,team_1,19,loss,0,srinidhi
7,12/29/2022,srinidhi,raghotham,ajay,prateek,21,16,isro match point,team_1,5,37,35 - 40,team_1,21,win,1,srinidhi
8,12/29/2022,srinidhi,raghotham,ajay,prateek,22,24,isro match point,team_2,2,46,> 45,team_1,22,loss,0,srinidhi
9,12/29/2022,srinidhi,raghotham,ajay,prateek,22,20,isro match point,team_1,2,42,40 - 45,team_1,22,win,1,srinidhi


In [21]:
head_2_head_df = player_matches[np.where(
    player_matches['belongs_to'] == 'team_1',
    np.logical_and.reduce([
        player_matches['partner'] == team_1[1],
        player_matches['team_2_player_1'].isin(team_2),
        player_matches['team_2_player_2'].isin(team_2),
    ]),
    np.logical_and.reduce([
        player_matches['partner'] == team_1[1],
        player_matches['team_1_player_1'].isin(team_2),
        player_matches['team_1_player_2'].isin(team_2),
    ])
)]

In [22]:
head_2_head_df

,date,team_1_player_1,team_1_player_2,team_2_player_1,team_2_player_2,points_team_1,points_team_2,venue,winner,margin,total_points_per_game,point_bins,belongs_to,player_team_points,result,is_win,partner
16,12/30/2022,nithin,raghotham,ajay,vinay,14,21,prakash badminton academy,team_2,7,35,35 - 40,team_1,14,loss,0,nithin
18,12/30/2022,nithin,raghotham,ajay,vinay,14,21,prakash badminton academy,team_2,7,35,35 - 40,team_1,14,loss,0,nithin
20,12/30/2022,nithin,raghotham,ajay,vinay,20,22,prakash badminton academy,team_2,2,42,40 - 45,team_1,20,loss,0,nithin
22,12/30/2022,nithin,raghotham,ajay,vinay,14,21,prakash badminton academy,team_2,7,35,35 - 40,team_1,14,loss,0,nithin
36,12/26/2022,nithin,raghotham,ajay,vinay,21,15,match point - gublaala,team_1,6,36,35 - 40,team_1,21,win,1,nithin
37,12/26/2022,nithin,raghotham,ajay,vinay,18,21,match point - gublaala,team_2,3,39,35 - 40,team_1,18,loss,0,nithin
38,12/26/2022,nithin,raghotham,ajay,vinay,16,21,match point - gublaala,team_2,5,37,35 - 40,team_1,16,loss,0,nithin
41,12/26/2022,nithin,raghotham,ajay,vinay,21,13,match point - gublaala,team_1,8,34,30 - 35,team_1,21,win,1,nithin


In [40]:
table = go.Table(
    header=dict(values=head_2_head_df.columns[:5]), 
    cells=dict(
        values=[head_2_head_df[i] for i in head_2_head_df.columns[:5]],
        
    )
)
fig = go.Figure(table)
fig

[['12/30/2022',
  'nithin',
  'raghotham',
  'ajay',
  'vinay',
  14,
  21,
  'prakash badminton academy',
  'team_2',
  7,
  35,
  '35 - 40',
  'team_1',
  14,
  'loss',
  0,
  'nithin'],
 ['12/30/2022',
  'nithin',
  'raghotham',
  'ajay',
  'vinay',
  14,
  21,
  'prakash badminton academy',
  'team_2',
  7,
  35,
  '35 - 40',
  'team_1',
  14,
  'loss',
  0,
  'nithin'],
 ['12/30/2022',
  'nithin',
  'raghotham',
  'ajay',
  'vinay',
  20,
  22,
  'prakash badminton academy',
  'team_2',
  2,
  42,
  '40 - 45',
  'team_1',
  20,
  'loss',
  0,
  'nithin'],
 ['12/30/2022',
  'nithin',
  'raghotham',
  'ajay',
  'vinay',
  14,
  21,
  'prakash badminton academy',
  'team_2',
  7,
  35,
  '35 - 40',
  'team_1',
  14,
  'loss',
  0,
  'nithin'],
 ['12/26/2022',
  'nithin',
  'raghotham',
  'ajay',
  'vinay',
  21,
  15,
  'match point - gublaala',
  'team_1',
  6,
  36,
  '35 - 40',
  'team_1',
  21,
  'win',
  1,
  'nithin'],
 ['12/26/2022',
  'nithin',
  'raghotham',
  'ajay',
  'vin